In [1]:
# atualizando o pip para a versão mais atual   
!pip install --upgrade pip

In [6]:
# instalando as depedencias necessárias
!pip install dlib==19.24.6


In [12]:
!pip install --no-cache-dir face_recognition


In [ ]:

# !pip install mediapipe

# !pip install msvc-runtime

!pip install --user opencv-python mediapipe deepface scikit-learn

In [17]:
!pip install tf_keras

  Using cached tf_keras-2.18.0-py3-none-any.whl.metadata (1.6 kB)
Using cached tf_keras-2.18.0-py3-none-any.whl (1.7 MB)


In [3]:
import cv2
from deepface import DeepFace
import mediapipe as mp
import json

In [8]:


VIDEO_PATH = "video_analise.mp4"  # Caminho para o vídeo
OUTPUT_REPORT = "report.json"
OUTPUT_VIDEO = "video_resultado.mp4"
FRAME_SKIP = 5  # Salta frames para acelerar o processamento

# Inicialização de módulos
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

def process_video(video_path):
    # Carrega o vídeo
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    print(f"Total Frames: {total_frames}, FPS: {fps}, Resolução: {width}x{height}")
    
    # Configuração do vídeo de saída
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(OUTPUT_VIDEO, fourcc, fps, (width, height))

    frame_count = 0
    faces_data = []
    emotions_data = []
    activities_data = []
    anomalies = []
    unique_emotions = set()
    unique_activities = set()

    pose = mp_pose.Pose()

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if frame_count % FRAME_SKIP == 0:
            # Processamento de reconhecimento facial
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            faces = detect_faces(frame)
            
            for (x, y, w, h) in faces:
                face_crop = frame[y:y+h, x:x+w]
                emotion = analyze_emotion(face_crop)
                emotions_data.append({"frame": frame_count, "emotion": emotion})
                unique_emotions.add(emotion)

                
                # Desenhar o rosto detectado
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                cv2.putText(frame, emotion, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            
            if len(faces) > 0:
                faces_data.append({"frame": frame_count, "faces_detected": len(faces)})
            
            # Detecção de atividades
            results = pose.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            if results.pose_landmarks:
                activity = detect_activity(results.pose_landmarks)
                activities_data.append({"frame": frame_count, "activity": activity})
                unique_activities.add(activity)
                mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
            
        # Escreve o frame processado no vídeo de saída
        out.write(frame)
        frame_count += 1

    cap.release()
    out.release()

    # Geração de relatório
    generate_report(total_frames, faces_data, emotions_data, activities_data, anomalies, unique_emotions, unique_activities)

def detect_faces(frame):
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)
    return faces

def analyze_emotion(face_crop):
    try:
        # analysis = DeepFace.analyze(face_crop, actions=['emotion'], enforce_detection=True, detector_backend='retinaface')
        analysis = DeepFace.analyze(face_crop, actions=['emotion'], enforce_detection=False)
        return analysis[0]['dominant_emotion']
    except Exception as e:
        return "unknown"

def detect_activity(landmarks):
    return "atividade_genérica"

def generate_report(total_frames, faces, emotions, activities, anomalies, unique_emotions, unique_activities):
    report = {
        "total_frames": total_frames,
        "faces_detected": len(faces),
        "emotions_analyzed": len(emotions),
        "unique_emotions": list(unique_emotions),
        "activities_detected": len(activities),
        "unique_activities": list(unique_activities),
        "anomalies_detected": len(anomalies),
        "frames_with_anomalies": anomalies
    }
    
    with open(OUTPUT_REPORT, "w") as f:
        json.dump(report, f, indent=4)
    print(f"Relatório salvo em {OUTPUT_REPORT}")

if __name__ == "__main__":
    process_video(VIDEO_PATH)


Total Frames: 3326, FPS: 30, Resolução: 1280x720


KeyboardInterrupt: 